# Capstone Project - The Battle of the Neighborhoods (Week 1)

## Data <a name="data"></a>

To solve the business problem, we need the following data:
* The latest licensed dogs and cats report by forward station area (FSA)
  (We will use cats and dog data as their number is much bigger than number of other pets)
* The historical licensed dogs and cats reports by forward station area
  (We need historical data to see the trends)
* List of Toronto neighborhoods with FSA
* Number and location of existing veterinary clinics in each neighborhood

We can get the dogs and cats reports from https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/licensed-dogs-and-cats-reports </br>
We will get the list of Toronto neighborhoods with FSA from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M using the BeautifulSoup library for Python to scrap the web page.</br>
Number and location of existing veterinary clinics in each neighborhood will be obtained using Foursquare API.

### Importing Data

First, we will import the cats and dogs data, clean it and explore it and then we will do the same for the eighborhoods data.

In [1]:
import pandas as pd
#Take only FSA and Total columns
df_cats_dogs_2020 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/b7e67ff9-7d58-4e8f-a4c7-3057416244e5/download/2020-to-september-30-dog-and-cat-licences-by-forward-sortation-area.xls', usecols=[0,3], skiprows=3)
df_cats_dogs_2019 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/dcf8954e-345f-4c5d-a039-945aca56e606/download/2019-dog-and-cat-licences-by-forward-sortation-area.xls', usecols=[0,3], skiprows=4)
df_cats_dogs_2018 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/61a31ba3-503a-4fce-ad23-e5005d3727fb/download/2018-dog-and-cat-licences-by-forward-sortation-area.xls', usecols=[0,3], skiprows=4)
df_cats_dogs_2017 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/dc91b1db-dff7-4dc2-88d1-6031faf5f941/download/by-forward-sortation-area-fsa-2017.xls', usecols=[0,3], skiprows=2)
df_cats_dogs_2016 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/932902b0-21e6-4a8e-9f2a-b4f3df5453bc/download/by-forward-sortation-area-fsa-2016.xls', usecols=[0,3], skiprows=2)
df_cats_dogs_2015 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/8ac516e4-9dd2-4d71-8a33-f74a9f2ce559/download/by-forward-sortation-area-fsa-2015.xls', usecols=[0,3], skiprows=2)
df_cats_dogs_2014 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/e9af254b-40be-4353-89c7-58a829bf2e67/download/by-forward-sortation-area-fsa-2014.xls', usecols=[0,3], skiprows=2)
df_cats_dogs_2013 = pd.read_excel ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c9ae19a-e23f-47d0-a813-742548d9a593/resource/42f9ad30-8230-4d01-9e61-86e1965ceccf/download/by-forward-sortation-area-fsa-2013.xls', usecols=[0,3], skiprows=3)



### Explore the cats and dogs data

Let's take a look at the most recent data:

In [2]:
df_cats_dogs_2020.head()

,FSA,Total
0,M1B,441
1,M1C,591
2,M1E,666
3,M1G,291
4,M1H,212


In [3]:
df_cats_dogs_2020.dtypes

FSA      object
Total     int64
dtype: object

In [4]:
df_cats_dogs_2019.dtypes

FSA      object
Total     int64
dtype: object

In [5]:
df_cats_dogs_2018.dtypes

FSA      object
Total     int64
dtype: object

In [6]:
df_cats_dogs_2017.dtypes

FSA      object
Total     int64
dtype: object

In [7]:
df_cats_dogs_2016.dtypes

Unnamed: 0    object
Total          int64
dtype: object

In [8]:
df_cats_dogs_2015.dtypes

Unnamed: 0    object
TOTAL          int64
dtype: object

In [9]:
df_cats_dogs_2014.dtypes

Unnamed: 0    object
TOTAL          int64
dtype: object

In [10]:
df_cats_dogs_2013.dtypes

Unnamed: 0    object
Total          int64
dtype: object

### Data Cleaning

We can see that some dataframes are missing the first column name and some have the "Total" column spelled in upper case. We need to fix that before merging the data.</br>
The idea is to merge the data into a single dataframe that contains the FSA code, totals for each year so that we can easily calculate the YoY % change in number of licensed pets, and we also need the columns to store latitude and longitude for each FSA.

In [11]:
df_cats_dogs_2016.rename(columns={ df_cats_dogs_2016.columns[0]: "FSA" }, inplace = True)
df_cats_dogs_2015.rename(columns={ df_cats_dogs_2015.columns[0]: "FSA" }, inplace = True)
df_cats_dogs_2014.rename(columns={ df_cats_dogs_2014.columns[0]: "FSA" }, inplace = True)
df_cats_dogs_2013.rename(columns={ df_cats_dogs_2013.columns[0]: "FSA" }, inplace = True)

df_cats_dogs_2020.rename(columns={'Total':'Total_2020'}, inplace = True)
df_cats_dogs_2019.rename(columns={'Total':'Total_2019'}, inplace = True)
df_cats_dogs_2018.rename(columns={'Total':'Total_2018'}, inplace = True)
df_cats_dogs_2017.rename(columns={'Total':'Total_2017'}, inplace = True)
df_cats_dogs_2016.rename(columns={'Total':'Total_2016'}, inplace = True)
df_cats_dogs_2015.rename(columns={'TOTAL':'Total_2015'}, inplace = True)
df_cats_dogs_2014.rename(columns={'TOTAL':'Total_2014'}, inplace = True)
df_cats_dogs_2013.rename(columns={'Total':'Total_2013'}, inplace = True)

Now we can merge dataframes:

In [12]:
data_frames = [df_cats_dogs_2013,df_cats_dogs_2014, df_cats_dogs_2015, df_cats_dogs_2016,df_cats_dogs_2017, df_cats_dogs_2018, df_cats_dogs_2019, df_cats_dogs_2020]

In [13]:
from functools import reduce
df_cats_dogs = reduce(lambda  left,right: pd.merge(left,right,on=['FSA'],
                                            how='outer'), data_frames)

In [14]:
df_cats_dogs.head()

,FSA,Total_2013,Total_2014,Total_2015,Total_2016,Total_2017,Total_2018,Total_2019,Total_2020
0,M0M,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,M1B,967.0,961.0,905.0,919.0,912.0,872.0,747.0,441.0
2,M1C,1354.0,1332.0,1227.0,1190.0,1072.0,1053.0,964.0,591.0
3,M1E,1621.0,1574.0,1494.0,1447.0,1430.0,1386.0,1218.0,666.0
4,M1G,709.0,654.0,655.0,609.0,605.0,572.0,494.0,291.0


Let's print the whole dataset:

In [15]:
for index, row in df_cats_dogs.iterrows():
    print(row["FSA"], row["Total_2013"], row["Total_2014"],row["Total_2015"],row["Total_2016"],row["Total_2017"],row["Total_2018"],row["Total_2019"],row["Total_2020"])

M0M 1.0 1.0 nan nan nan nan nan nan
M1B 967.0 961.0 905.0 919.0 912.0 872.0 747.0 441.0
M1C 1354.0 1332.0 1227.0 1190.0 1072.0 1053.0 964.0 591.0
M1E 1621.0 1574.0 1494.0 1447.0 1430.0 1386.0 1218.0 666.0
M1G 709.0 654.0 655.0 609.0 605.0 572.0 494.0 291.0
M1H 545.0 545.0 512.0 500.0 464.0 437.0 405.0 212.0
M1J 674.0 642.0 650.0 622.0 618.0 594.0 542.0 314.0
M1K 1042.0 1033.0 1003.0 1059.0 1107.0 954.0 834.0 502.0
M1L 833.0 823.0 834.0 793.0 809.0 773.0 686.0 385.0
M1M 976.0 942.0 924.0 852.0 877.0 835.0 700.0 427.0
M1N 1333.0 1312.0 1212.0 1256.0 1245.0 1143.0 995.0 633.0
M1P 826.0 808.0 822.0 815.0 798.0 749.0 640.0 382.0
M1R 791.0 793.0 758.0 742.0 755.0 678.0 595.0 343.0
M1S 677.0 663.0 671.0 630.0 615.0 621.0 521.0 327.0
M1T 598.0 550.0 571.0 556.0 590.0 583.0 513.0 320.0
M1V 724.0 668.0 675.0 663.0 615.0 620.0 515.0 331.0
M1W 736.0 678.0 644.0 639.0 670.0 613.0 545.0 385.0
M1X 161.0 171.0 155.0 187.0 148.0 146.0 155.0 81.0
M2H 523.0 525.0 517.0 538.0 529.0 492.0 433.0 249.0
M2J 8

There are null values but they can be dropped as they exist on FSA codes with very small numbers of cats and dogs (1 to 4). These numbers are outliers and should be removed:

In [16]:
# removing null values
df_cats_dogs = df_cats_dogs.dropna()

Let's print the data again:

In [17]:
for index, row in df_cats_dogs.iterrows():
    print(row["FSA"], row["Total_2013"], row["Total_2014"],row["Total_2015"],row["Total_2016"],row["Total_2017"],row["Total_2018"],row["Total_2019"],row["Total_2020"])

M1B 967.0 961.0 905.0 919.0 912.0 872.0 747.0 441.0
M1C 1354.0 1332.0 1227.0 1190.0 1072.0 1053.0 964.0 591.0
M1E 1621.0 1574.0 1494.0 1447.0 1430.0 1386.0 1218.0 666.0
M1G 709.0 654.0 655.0 609.0 605.0 572.0 494.0 291.0
M1H 545.0 545.0 512.0 500.0 464.0 437.0 405.0 212.0
M1J 674.0 642.0 650.0 622.0 618.0 594.0 542.0 314.0
M1K 1042.0 1033.0 1003.0 1059.0 1107.0 954.0 834.0 502.0
M1L 833.0 823.0 834.0 793.0 809.0 773.0 686.0 385.0
M1M 976.0 942.0 924.0 852.0 877.0 835.0 700.0 427.0
M1N 1333.0 1312.0 1212.0 1256.0 1245.0 1143.0 995.0 633.0
M1P 826.0 808.0 822.0 815.0 798.0 749.0 640.0 382.0
M1R 791.0 793.0 758.0 742.0 755.0 678.0 595.0 343.0
M1S 677.0 663.0 671.0 630.0 615.0 621.0 521.0 327.0
M1T 598.0 550.0 571.0 556.0 590.0 583.0 513.0 320.0
M1V 724.0 668.0 675.0 663.0 615.0 620.0 515.0 331.0
M1W 736.0 678.0 644.0 639.0 670.0 613.0 545.0 385.0
M1X 161.0 171.0 155.0 187.0 148.0 146.0 155.0 81.0
M2H 523.0 525.0 517.0 538.0 529.0 492.0 433.0 249.0
M2J 847.0 939.0 908.0 901.0 954.0 909.0 8

In [18]:
df_cats_dogs = df_cats_dogs.set_index('FSA').reset_index()
df_cats_dogs.head()

,FSA,Total_2013,Total_2014,Total_2015,Total_2016,Total_2017,Total_2018,Total_2019,Total_2020
0,M1B,967.0,961.0,905.0,919.0,912.0,872.0,747.0,441.0
1,M1C,1354.0,1332.0,1227.0,1190.0,1072.0,1053.0,964.0,591.0
2,M1E,1621.0,1574.0,1494.0,1447.0,1430.0,1386.0,1218.0,666.0
3,M1G,709.0,654.0,655.0,609.0,605.0,572.0,494.0,291.0
4,M1H,545.0,545.0,512.0,500.0,464.0,437.0,405.0,212.0


The data looks good now.</br>
The next step is to add geo locations so that we can visualize data:

### Explore the cats and dogs data

#### Let's get the geo locations for each FSA and add it to the dataframe

First, add the Latitude and Longitude columns:

In [19]:
import numpy as np
df_cats_dogs.insert(loc=9, column = 'Latitude', value = np.NaN)
df_cats_dogs.insert(loc=10, column = 'Longitude', value = np.NaN)

In [20]:
!pip install geocoder
import os
import geocoder
import requests
GOOGLE_API_KEY = '' 
print("Libraries Imported.")

Libraries Imported.


In [21]:
def get_lat_long(address):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address}&key={api_key}"
    request = requests.get(endpoint)
    if request.status_code not in range(200, 299):
        return None, None
    try:
        results = request.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return lat, lng

In [22]:
for index, row in df_cats_dogs.iterrows():
    lat, lon = get_lat_long(row['FSA'] + ", Toronto")
    df_cats_dogs.loc[index, 'Latitude'] = lat
    df_cats_dogs.loc[index, 'Longitude'] = lon
    print('.', sep=' ', end='', flush=True)
df_cats_dogs.head()    

................................................................................................

,FSA,Total_2013,Total_2014,Total_2015,Total_2016,Total_2017,Total_2018,Total_2019,Total_2020,Latitude,Longitude
0,M1B,967.0,961.0,905.0,919.0,912.0,872.0,747.0,441.0,43.806686,-79.194353
1,M1C,1354.0,1332.0,1227.0,1190.0,1072.0,1053.0,964.0,591.0,43.784535,-79.160497
2,M1E,1621.0,1574.0,1494.0,1447.0,1430.0,1386.0,1218.0,666.0,43.763573,-79.188711
3,M1G,709.0,654.0,655.0,609.0,605.0,572.0,494.0,291.0,43.770992,-79.216917
4,M1H,545.0,545.0,512.0,500.0,464.0,437.0,405.0,212.0,43.773136,-79.239476


In [23]:
df_cats_dogs.shape

(96, 11)

#### Let's see where the cats and dogs are on the map:
We will display the most recent data (2020)

In [24]:
#!pip install folium
import folium

First, let's find geo location of downtown Toronto using Google Maps geocoding API:

In [25]:
address = 'Downtown Toronto, ON, Canada'
latitude_toronto, longitude_toronto = get_lat_long(address)
toronto_downtown = [latitude_toronto, longitude_toronto]
print("Downtown Toronto","latitude",latitude_toronto, "& " "longitude" ,longitude_toronto)

Downtown Toronto latitude 43.6548046 & longitude -79.3883031


In [26]:
map_toronto = folium.Map(location=toronto_downtown, zoom_start=13)
folium.Marker(toronto_downtown, popup='Toronto').add_to(map_toronto)
for i in range(0,len(df_cats_dogs)):  
    folium.Circle(
      location=[df_cats_dogs.iloc[i]['Latitude'], df_cats_dogs.iloc[i]['Longitude']],
      popup=df_cats_dogs.iloc[i]['FSA'] + ":" + str(df_cats_dogs.iloc[i]['Total_2020']),
      radius=float(df_cats_dogs.iloc[i]['Total_2020'])/1.5,
      color='crimson',
      fill=True,
      fill_color='crimson'
    ).add_to(map_toronto)
    folium.Marker(toronto_downtown, popup='Toronto').add_to(map_toronto)
map_toronto

The data looks good. Now, we need to import neighborhood data.

### Scrapping the data from a Wikipedia page that contains a table with postal codes and neighborhood names

We will use the BeautifulSoup to extract the data from a table on Wikipedia.

In [27]:
import requests # library to handle requests
from html_table_extractor.extractor import Extractor
from bs4 import BeautifulSoup
from IPython.display import display_html

In [28]:
#Scrap of the data of Wikipedia and explore the data
data_source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data_source, 'html.parser')
extractor = Extractor(soup.table) 
neighborhoods_list=extractor.return_list()
display_html(str(soup.table),raw=True)

Postal Code,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M8A,Not assigned,Not assigned
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"


In [29]:
df = pd.DataFrame (neighborhoods_list,columns=['Postal Code','Borough','Neighborhood'])
df = df.iloc[1:]
df.head()

,Postal Code,Borough,Neighborhood
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### Clean the neighborhood data
* Remove \n at the end of each cell value
* Drop the rows that have the value of Borough equal to "Not assigned"
* Rename the "Postal Code" column to "FSA"

In [30]:
df_toronto_Neighborhoods = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)

df_toronto_Neighborhoods.reset_index(inplace=True)

#Removing \n
for index, row in df_toronto_Neighborhoods.iterrows():
    row['Postal Code'] = row['Postal Code'].replace('\n','')
    row['Borough'] = row['Borough'].replace('\n','')
    row['Neighborhood'] = row['Neighborhood'].replace('\n','')
    if row['Neighborhood'].find(",")>0:
        row['Neighborhood'] = row['Neighborhood'][0:row['Neighborhood'].replace('\n','').find(",")]
#Dropping of rows if the value of Borough is "Not assigned"
df_toronto_Neighborhoods = df_toronto_Neighborhoods[df_toronto_Neighborhoods['Borough'] != 'Not assigned']
df_toronto_Neighborhoods.rename(columns={'Postal Code':'FSA'},inplace=True)
df_toronto_Neighborhoods.head()

,FSA,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park


In [31]:
df_toronto_Neighborhoods.shape

(103, 3)

In [32]:
null_rows = df_toronto_Neighborhoods[df_toronto_Neighborhoods['FSA'].isnull()]
null_rows

,FSA,Borough,Neighborhood


The data looks good. There are no null values. We can now merge the pets and neighborhoods dataframes

In [33]:
df_neighborhoods_pets = pd.merge(df_cats_dogs, df_toronto_Neighborhoods, on='FSA',  how='left')
df_neighborhoods_pets.head()

,FSA,Total_2013,Total_2014,Total_2015,Total_2016,Total_2017,Total_2018,Total_2019,Total_2020,Latitude,Longitude,Borough,Neighborhood
0,M1B,967.0,961.0,905.0,919.0,912.0,872.0,747.0,441.0,43.806686,-79.194353,Scarborough,Malvern
1,M1C,1354.0,1332.0,1227.0,1190.0,1072.0,1053.0,964.0,591.0,43.784535,-79.160497,Scarborough,Rouge Hill
2,M1E,1621.0,1574.0,1494.0,1447.0,1430.0,1386.0,1218.0,666.0,43.763573,-79.188711,Scarborough,Guildwood
3,M1G,709.0,654.0,655.0,609.0,605.0,572.0,494.0,291.0,43.770992,-79.216917,Scarborough,Woburn
4,M1H,545.0,545.0,512.0,500.0,464.0,437.0,405.0,212.0,43.773136,-79.239476,Scarborough,Cedarbrae


In [34]:
df_neighborhoods_pets.shape

(96, 13)

#### Let's visualize the neighborhoods data 

In [46]:
map_toronto = folium.Map(location=toronto_downtown, zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_neighborhoods_pets['Latitude'], df_neighborhoods_pets['Longitude'], df_neighborhoods_pets['Borough'], df_neighborhoods_pets['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

We can now merge pets and Neighborhoods data:

### Using Foursquare API to get veterinary clinics information

#### Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = ''
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [37]:
def get_venues( category_id, lat, lon, client_id, client_secret, version, radius=500, limit=100):

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lon, 
            radius, 
            limit,
            category_id)
    try:           
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                    item['venue']['name'],
                    item['venue']['categories'][0]['name'],
                    item['venue']['location']['lat'], 
                    item['venue']['location']['lng']
                  ) for item in results]        
    except:
        pass
    return venues

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's get all veterinary venues in radius of 10km of downtown Toronto.
The id of the veterinary venue in Foursquare is 4d954af4a243a5684765b473 </br>
We need the latitude and longitude of downtown Toronto which we already found in the previous section. It is (43.6548046,-79.3883031)

In [39]:
import numpy as np
all_veterinary_venues = get_venues("4d954af4a243a5684765b473", 43.6548046,-79.3883031, CLIENT_ID, CLIENT_SECRET, VERSION, radius=10000, limit=500)
arr = np.array(all_veterinary_venues)
all_veterinary_venues_df = pd.DataFrame(arr, columns = ['Venue Id', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude'])

In [40]:
all_veterinary_venues_df.shape

(38, 5)

In [41]:
all_veterinary_venues_df.head()

,Venue Id,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,4ae06ddef964a5203b7f21e3,Toronto Humane Society,Animal Shelter,43.65769802359708,-79.3565828808708
1,4b81644af964a52049a330e3,VEC Veterinary Emergency Clinic,Veterinarian,43.67397762493825,-79.38973071994448
2,4bd1dc14046076b0f0a77271,Spadina Animal Hospital,Veterinarian,43.64722300237428,-79.39565838078587
3,4db09bf70437a93f7f770755,Wellesley Animal Hospital,Veterinarian,43.66494103924063,-79.38458123431751
4,516ff685e4b03e954dae2613,Sherbourne Animal Hospital,Veterinarian,43.653999,-79.369174


In [42]:
all_veterinary_venues_df.dtypes

Venue Id           object
Venue Name         object
Venue Category     object
Venue Latitude     object
Venue Longitude    object
dtype: object

We need to convert Venue Latitude and Venue Longitude to float:

In [43]:
all_veterinary_venues_df[['Venue Latitude', 'Venue Longitude']] = all_veterinary_venues_df[['Venue Latitude', 'Venue Longitude']].apply(pd.to_numeric) 

In [44]:
all_veterinary_venues_df.head()

,Venue Id,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,4ae06ddef964a5203b7f21e3,Toronto Humane Society,Animal Shelter,43.657698,-79.356583
1,4b81644af964a52049a330e3,VEC Veterinary Emergency Clinic,Veterinarian,43.673978,-79.389731
2,4bd1dc14046076b0f0a77271,Spadina Animal Hospital,Veterinarian,43.647223,-79.395658
3,4db09bf70437a93f7f770755,Wellesley Animal Hospital,Veterinarian,43.664941,-79.384581
4,516ff685e4b03e954dae2613,Sherbourne Animal Hospital,Veterinarian,43.653999,-79.369174


Let's display the venues on the map:

In [45]:
map_toronto_venues = folium.Map(location=toronto_downtown, zoom_start=12)
for lat, lng, venue in zip(all_veterinary_venues_df['Venue Latitude'], all_veterinary_venues_df['Venue Longitude'], all_veterinary_venues_df['Venue Name']):
    label = '{}'.format(venue)
    label = folium.Popup(venue, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_venues)  
map_toronto_venues

#### Calculate the distance from each neighborhood to all venues

We will need the distance from neighborhoods to venues as this will be one of the parameters for clustering the neighborhoods and finding the optimal location for a new venue.
we will use geopy library to measure distances.

In [60]:
from geopy.distance import distance
df_venue_distances = pd.DataFrame(columns = ['FSA','Venue Id','Distance'])
i=0
for lat_n, lng_n, fsa in zip(df_neighborhoods_pets['Latitude'], df_neighborhoods_pets['Longitude'], df_neighborhoods_pets['FSA']):
    for lat_v, lng_v, id_v in zip(all_veterinary_venues_df['Venue Latitude'], all_veterinary_venues_df['Venue Longitude'], all_veterinary_venues_df['Venue Id']):
        i = i + 1
        d = distance([lat_n, lng_n], [lat_v, lng_v]).m
        df_venue_distances.loc[i] = [fsa]+[id_v]+[d]

  


In [61]:
df_venue_distances = df_venue_distances.sort_values(by=['FSA','Distance'])

In [63]:
df_venue_min_distances = df_venue_distances.groupby('FSA').head(3)

In [65]:
df_venue_min_distances.head(9)

,FSA,Venue Id,Distance
37,M1B,4b6db982f964a520dc892ce3,14595.586704
34,M1B,4bab7c25f964a52005ad3ae3,15350.860947
35,M1B,4b749ea3f964a5202ee82de3,15771.598375
72,M1C,4bab7c25f964a52005ad3ae3,15152.220724
73,M1C,4b749ea3f964a5202ee82de3,15267.851001
75,M1C,4b6db982f964a520dc892ce3,15965.763664
110,M1E,4bab7c25f964a52005ad3ae3,11900.508184
111,M1E,4b749ea3f964a5202ee82de3,12021.275039
113,M1E,4b6db982f964a520dc892ce3,13118.842505


We have all the data that we need to solve the business problem:

*  Licensed dogs and cats reports by forward station area for 2013 to September 2020.
*  Toronto neighborhood list with FSA codes
*  All the venues in Toronto area categorized as veterinary services in Foursquare

We cleaned the data (removed outliers and null values, restored the missing column names).</br>
Then we used Google geocoding API to get the latitude and longitude for each FSA.</br>
We explored the data, displayed it on the map.</br>
Finally, we used Geopy to calculate the distance from neighborhoods to the venues.>/br>historical licensed dogs and cats reports by forward station area</br>
This concluded the Data section.

